In [1]:
import pandas as pd
import numpy as np

In [2]:
import sys
sys.path.append("../code/")

In [4]:
from regional_average_contribution import contribution

In [10]:
from create_matrix import *

from tensor_custom_core import *
from create_matrix import *
from tensor_custom_core import *
appliance_index = {appliance: APPLIANCES_ORDER.index(appliance) for appliance in APPLIANCES_ORDER}

APPLIANCES = ['fridge', 'hvac', 'wm', 'mw', 'oven', 'dw']
region = "SanDiego"
year = 2014

import os


In [11]:
def get_tensor_appliance(df, dfc, appliance):
    start, stop = 1, 13
    energy_cols = np.array(
        [['%s_%d' % (appliance, month) for month in range(start, stop)] ]).flatten()
    static_cols = ['area', 'total_occupants', 'num_rooms']
    static_df = df[static_cols]
    static_df = static_df.div(static_df.max())
    weather_values = np.array(dds[2014][region][start - 1:stop - 1]).reshape(-1, 1)

    dfc = df.copy()

    df = dfc[energy_cols]
    col_max = df.max().max()
    col_min = df.min().min()
    # df = (1.0 * (df - col_min)) / (col_max - col_min)
    tensor = df.values.reshape((len(df), 1, stop - start))
    M, N, O = tensor.shape
    return tensor

In [17]:
def get_tensor(df):
    start, stop = 1, 13
    energy_cols = np.array(
        [['%s_%d' % (appliance, month) for month in range(start, stop)] for appliance in APPLIANCES_ORDER]).flatten()
    static_cols = ['area', 'total_occupants', 'num_rooms']
    static_df = df[static_cols]
    static_df = static_df.div(static_df.max())

    #dfc = df.copy()

    df = df[energy_cols]
    col_max = df.max().max()
    col_min = df.min().min()
    # df = (1.0 * (df - col_min)) / (col_max - col_min)
    tensor = df.values.reshape((len(df), 7, stop - start))
    M, N, O = tensor.shape
    return tensor

In [18]:
sd_df, sd_dfc = create_matrix_single_region("SanDiego", year)
sd_tensor = get_tensor(sd_df)

In [20]:
sd_tensor.shape

(39, 7, 12)

In [52]:
def get_regional_average(appliance, test_region='Austin', year=2014):
    try:
        contri_austin_appliance = contribution[test_region]['Cooling'][appliance]
        if appliance=="hvac":
            start, stop=5, 11
        else:
            start, stop=1, 13
        test_df, _ = create_matrix_single_region(test_region, year)
        #test_df=test_df[(test_df.full_agg_available==1)&(test_df.md_available==1)]

        test_df_appliance=test_df.ix[test_df[['%s_%d' %(appliance,month) for month in range(start,stop)]].index]
        pred_df = test_df_appliance[['aggregate_%d' %month for month in range(start, stop)]]*contri_austin_appliance
        pred_df.columns = ['%s_%d' %(appliance, month) for month in range(start, stop)]
        #gt_df = test_df_appliance[['%s_%d' %(appliance, month) for month in range(start, stop)]]
        #gt_df.columns = pred_df.columns
        #error = (gt_df-pred_df).abs().div(gt_df).mul(100)
        return pred_df
    except Exception, e: 
        print e
        
        return np.NaN

In [53]:
APPLIANCES_ORDER.index("hvac")

1

In [54]:
pred = {}
for appliance in APPLIANCES_ORDER[1:]:
    try:
        pred[appliance] = get_regional_average(appliance,"SanDiego", 2014)
    except Exception, e:
        print(e)

'mw'
'oven'


/Users/nipunbatra/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  # This is added back by InteractiveShellApp.init_path()


In [68]:
from common import compute_rmse_fraction
error_ra = {}
for appliance in pred:
    try:
        error_ra[appliance] = compute_rmse_fraction(appliance, pd.DataFrame(pred[appliance]), 'SanDiego')[2]
    except:
        pass

In [69]:
pd.Series(error_ra)

dw         1.887536
fridge     6.562542
hvac      21.254885
wm         0.655845
dtype: float64

Comparison with other results with 6% data

In [71]:
import pickle
out = pickle.load(open("../predictions/lr-tf-Austin-SanDiego-all.pkl", "r"))

In [67]:
mapping = {2:'Our TF', 4:'STF'}
for case in [2, 4]:
    for constant_use in ['True', 'False']:
    
        for static in ['False','True']:
            for transfer in ['normal','transfer']:
                algo = "{}_{}_{}_{}".format(transfer, constant_use, static, mapping[case])
                error = out[case][constant_use][static][transfer]

{2: {'False': {'False': {'normal': {6.0: dw         2.500857
     fridge     8.679555
     hvac      24.788731
     mw         2.508361
     oven       5.233910
     wm         6.460755
     dtype: float64, 7.0: dw         2.500857
     fridge     8.679555
     hvac      24.788731
     mw         2.508361
     oven       5.233910
     wm         6.460755
     dtype: float64, 8.0: dw         2.500857
     fridge     8.679555
     hvac      24.788731
     mw         2.508361
     oven       5.233910
     wm         6.460755
     dtype: float64, 9.0: dw         2.523779
     fridge     9.607099
     hvac      23.178130
     mw         2.650495
     oven       3.652054
     wm         5.363489
     dtype: float64, 10.0: dw         2.523779
     fridge     9.607099
     hvac      23.178130
     mw         2.650495
     oven       3.652054
     wm         5.363489
     dtype: float64, 15.0: dw         2.196077
     fridge     8.860323
     hvac      21.118005
     mw         2.049913
     ov

In [95]:
out[2]['True']['False']['transfer'][6]

dw         1.859048
fridge     9.269688
hvac      19.295316
mw         2.173627
oven       2.350500
wm         0.828383
dtype: float64

In [110]:
pred['wm'].head()

,wm_1,wm_2,wm_3,wm_4,wm_5,wm_6,wm_7,wm_8,wm_9,wm_10,wm_11,wm_12
54,NaN,NaN,2.207017,2.729303,3.097376,3.197323,4.361391,4.469005,NaN,NaN,NaN,NaN
203,NaN,NaN,NaN,NaN,NaN,4.371343,6.849067,5.819301,4.474702,2.416179,2.057902,2.903334
527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.571611,12.500868,NaN,NaN,NaN
1450,4.298196,4.385717,4.030897,4.593837,8.613099,10.085893,9.174286,11.768164,10.981251,9.060317,5.936323,8.014442
1524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.326741,11.574678,5.784237,2.513506,2.960005


In [118]:
%matplotlib inline

In [120]:
a = sd_df[['wm_{}'.format(month) for month in range(1, 13)]]
b = sd_df[['aggregate_{}'.format(month) for month in range(1, 13)]]
b.columns = a.columns
(a/b).describe()

,wm_1,wm_2,wm_3,wm_4,wm_5,wm_6,wm_7,wm_8,wm_9,wm_10,wm_11,wm_12
count,7.000000,6.000000,9.000000,8.000000,8.000000,13.000000,13.000000,10.000000,8.000000,8.000000,8.000000,7.000000
mean,0.012235,0.013421,0.017172,0.014460,0.008841,0.013015,0.009130,0.007664,0.007296,0.009008,0.010840,0.008800
std,0.005447,0.005056,0.012588,0.009198,0.004657,0.007013,0.004300,0.002358,0.003743,0.003374,0.003169,0.003786
min,0.005250,0.008503,0.003342,0.005562,0.002748,0.004902,0.003988,0.003844,0.003488,0.003675,0.006932,0.003925
25%,0.007533,0.009210,0.008449,0.008212,0.005614,0.007995,0.006966,0.006619,0.004772,0.006946,0.008059,0.006070
50%,0.013456,0.012297,0.016198,0.012525,0.007408,0.011550,0.007899,0.007055,0.006139,0.008759,0.011327,0.008249
75%,0.016369,0.017027,0.017450,0.017394,0.013840,0.016151,0.010736,0.009559,0.008741,0.010727,0.012812,0.012186
max,0.019134,0.020643,0.043261,0.033317,0.014502,0.027306,0.021045,0.011532,0.013560,0.014164,0.014885,0.012913


In [102]:
sd_df[['wm_{}'.format(month) for month in range(1, 13)]].unstack().dropna().index

MultiIndex(levels=[[u'wm_1', u'wm_2', u'wm_3', u'wm_4', u'wm_5', u'wm_6', u'wm_7', u'wm_8', u'wm_9', u'wm_10', u'wm_11', u'wm_12'], [54, 203, 527, 1450, 1524, 1731, 2031, 2354, 2606, 3687, 3864, 3938, 4083, 4095, 4329, 4495, 4761, 4934, 5909, 5938, 6268, 6377, 6429, 6497, 6547, 7062, 7114, 7409, 7639, 8061, 8342, 8574, 8733, 9213, 9370, 9585, 9612, 9775, 9836]],
           labels=[[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11], [11, 14, 20, 21, 31, 33, 37, 11, 14, 21, 31, 33, 37, 0, 11, 14, 18, 21, 31, 32, 33, 37, 11, 14, 18, 21, 31, 32, 33, 37, 11, 14, 21, 28, 31, 32, 33, 37, 0, 6, 7, 11, 12, 21, 27, 28, 31, 32, 33, 35, 37, 0, 6, 7, 11, 12, 21, 27, 28, 31, 32, 33, 35, 37, 11, 12, 21, 27, 28, 31, 32, 33, 35

In [107]:
len(sd_df[['wm_{}'.format(month) for month in range(1, 13)]].unstack().dropna())

105

In [108]:
39*12

468

In [109]:
105/468.

0.22435897435897437